In [ ]:
from extract.common import credentials
from extract.common import config

In [ ]:
import pymongo
from bson.json_util import dumps

host = config()['mongodb']['host']
user = credentials()['mongodb']['user']
password = credentials()['mongodb']['password']

class SaveProjects(object):
    def __init__(self):
        uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
        client = pymongo.MongoClient(uri)
        self._db = client[config()['mongodb']['db']['name']]
    
    def save(self, projects):
        self._db.project.insert_many(projects)

In [ ]:
save_project = SaveProjects()

In [ ]:
save_project._db.project.delete_many({})

In [ ]:
save_project._db.project.estimated_document_count()
[i for i in save_project._db.project.find()]

In [ ]:
save_project.save([{"a":1},{"b":2}])

In [ ]:
from calculator.cumplo import predict_by_year

In [ ]:
vehicles = [{
    "time": 60,
    "yield": 0.18,
    "capital": 0
} for i in range(30)]
vehicles[0]['capital'] = 5000
prediction = predict_by_year(vehicles)
prediction

In [ ]:
import pandas as pd
import numpy as np
import datetime

import iplotter
from IPython.core.display import HTML
from visualize.chart import Chart

In [ ]:
today = datetime.datetime.now()
time = np.array([today + datetime.timedelta(i['time']) for i in prediction['time_serie']])
capital = np.array([i['capital'] for i in prediction['time_serie']])
return_ = np.array([i['return']['return'] for i in prediction['time_serie']])
ts = pd.Series(return_, index=time)
ts = ts.cumsum()
df = pd.DataFrame({"return": return_}, index = time )
df = df.cumsum()
df['capital'] = pd.Series(capital, index=time)

invested = (df['capital']-df['return']).tolist()
profits = df['return'].tolist()
x = df.index.strftime('%Y-%m-%d').tolist()

In [ ]:
chart = Chart().stacked_area(x, profits, invested)
HTML(Chart.html)
c3_plotter = iplotter.C3Plotter()
c3_plotter.plot(chart)